In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy.spatial import distance

# define spatial weight function
def sw_dist(dist, tsw, td):
    spatial_weight = np.exp(dist * np.log(tsw) / td)
    return spatial_weight

df = pd.read_csv('./Data/LSD_5179_CSV.txt')

# 아래 parameter 값들은 시간적 , 공간적 범위를 결정
tsw = 0.1; td = 3000  # tsw = threshold spatial weight; td = threshold distance
ttw = 0.1; ttd = 10   # ttw = threshold temporal weight; ttd = threshold temporal distance

# 공간가중치 행렬(od_sw_mat) 생성 : origin destination spatial weight matrix
od_x, od_y = np.array(df['px']), np.array(df['py'])
arr_od = np.stack([od_x, od_y], 1)
od_dist_matrix = distance.cdist(arr_od, arr_od)
od_sw_mat = sw_dist(od_dist_matrix, tsw, td)

# 시간가중치 행렬(ft_tw_mat) 생성 : from to temporal weight matrix
con_dt = np.array(df['확진일'])
arr_con_dt = np.stack([con_dt, con_dt], 1)
ft_dist_matrix = distance.cdist(arr_con_dt, arr_con_dt)
ft_tw_mat = sw_dist(ft_dist_matrix, ttw, ttd)

#아르마다 행렬 곱을 적용하여 시공간가중치 행렬(= 공간가중치 * 시간가중치) 생성 : spatial temporal weigh matrix
stw_mat = od_sw_mat * ft_tw_mat

#모든 발생지점을 기준으로 시공간가중 발생지점의 개수 행렬 (st_cnt_matrix) 계산 
st_cnt_matrix = np.sum(stw_mat, axis=0)

#시공간 가중행렬 적용하지 않은 발생시점 개수 행렬 : 시간 ttd 범위 내 & 공간 td 범위 내  
binary_cnt_spatial = np.where(od_dist_matrix <= td, 1, 0)
binary_cnt_temporal = np.where(ft_dist_matrix <= ttd, 1, 0)
binary_cnt_spatiotemporal = binary_cnt_spatial * binary_cnt_temporal
binary_cnt_spatiotemporal = np.sum(binary_cnt_spatiotemporal, axis=0)

df_id_x_y_date_count = pd.DataFrame()
df_id_x_y_date_count['차수'] = df['차수']
df_id_x_y_date_count['X'] = df['px']
df_id_x_y_date_count['Y'] = df['py']
df_id_x_y_date_count['Cnf_Dt'] = df['확진일']
df_id_x_y_date_count['CNT_BN'] = binary_cnt_spatiotemporal
df_id_x_y_date_count['CNT_ST'] = st_cnt_matrix

# GeoDataFrame을 Shapefile로 저장
#*******************************************************************************************************************************
gdf_occurrences = gpd.GeoDataFrame(df_id_x_y_date_count[['차수', 'X', 'Y', 'Cnf_Dt', 'CNT_BN', 'CNT_ST']], 
                       geometry=gpd.points_from_xy(df_id_x_y_date_count.X, df_id_x_y_date_count.Y))

parameters = 'counts(' + str(td) + '_' + str(tsw) + '_' + str(ttd) + '_' + str(ttw) + ')'

gdf_occurrences.crs= "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
gdf_occurrences.to_file('./Data/' +  parameters + '.shp', encoding='utf-8', driver='ESRI Shapefile')
#*******************************************************************************************************************************

df_id_x_y_date_count

,차수,X,Y,Cnf_Dt,CNT_BN,CNT_ST
0,1,9.033455e+05,1.853359e+06,45219,9,2.555407
1,2,9.366843e+05,1.874491e+06,45220,6,1.848566
2,3,9.537968e+05,1.894297e+06,45220,1,1.003053
3,4,9.022439e+05,1.852217e+06,45220,10,2.347358
4,5,9.241720e+05,1.972450e+06,45221,3,1.191622
...,...,...,...,...,...,...
102,103,9.795019e+05,1.743525e+06,45247,1,1.000000
103,104,9.964350e+05,1.858184e+06,45248,2,1.103350
104,105,9.082597e+05,1.722622e+06,45248,3,1.252946
105,106,1.080462e+06,2.043316e+06,45249,1,1.009208
